## Reprodução feita por Carlos Vinicius 
### Baseada nos pseudocódigos do artigo "MinSearch: An Efficient Algorithm for Similarity Search under Edit Distance"

In [126]:
from random import random
class RandomHash:
    def __init__(self):
        self.table = dict()
    def of(self, x):
        # Associa para cada string q-gram um valor real aleatório entre 0 e 1
        if x not in self.table:
            self.table[x] = random()
        return self.table[x]

In [127]:
def build_array(s, q = 2):
    """
    build an array A with |s | − q + 1 elements, where the i-th element
    A[i] = F (si..i+q−1), where F : Σq → (0, 1) is a random hash function
    """
    from random import random
    F = RandomHash()
    A = []
    for i in range(len(s)-q+1):
        A.append(F.of(s[i:i+q]))
    return A    

In [128]:
build_array('amigo', 3)

[0.7482187203239291, 0.5988346735122353, 0.6336472606984733]

In [129]:
def Rank(s, r, q = 2):
    """
    Input: input string s; minimum rank r
    Output: rank array R = {(pi, Ri )}: pi denotes the index of the letter in
    s corresponding to the i-th pair, and Ri is its rank
    """
    A = build_array(s, q)
    R = [(0, float('inf'))] # the first character of s has ranking infinity
    for i in range(len(A)):
        x = 1
        while i-x >= 0 and i+x < len(A):
            if A[i] < min(A[i+x], A[i-x]):
                x += 1
            else:
                break
        if x > r:
            R.append((i, x-1))
    R.append((len(s)-q, float('inf')))
    return R   

In [130]:
Rank('ACGTTCGACTGGTTAG', 1)

[(0, inf), (1, 1), (6, 6), (11, 3), (14, inf)]

In [131]:
def Partition(s, R, start, end):
    """
    Input: input string s; rank array R = {(pi, Ri )} of s; two indices start and end
    Output: set of partitions P = {(ssub , l)}, where (ssub , l) denotes a
    substring ssub with level l
    """
    if end <= start + 1:
        return []
    maxR = float('-inf')
    M = []
    i = start + 1
    while i < end:
        if R[i][1] > maxR:
            maxR = R[i][1]
            M = [i]
        if R[i][1] == maxR:
            M.append(i)
        i += 1
    P = []
    M = [start] + M + [end]
    M.sort()
    for j in range(len(M)-1):
        u = M[j]
        v = M[j+1]
        pu = R[u][0]
        pv_minus_1 = R[v-1][0]
        pv = R[v][0]
        P.append((s[pu:pv_minus_1 + 1], min(R[u][1], R[v][1])))
        P = P+(Partition(s, R, u, v))
    return P

In [132]:
R = Rank('ACGTTCGACTGGTTAG', 1)
Partition('ACGTTCGACTGGTTAG', R, 0, len(R)-1)

[('ACGTTCGA', 4),
 ('ACGTT', 2),
 ('A', 1),
 ('', 1),
 ('G', 1),
 ('T', 1),
 ('', 2),
 ('A', 2),
 ('', 4),
 ('G', 4)]

In [133]:
from random import randint
from collections import defaultdict
class RandomHashInt:
    def __init__(self):
        self.table = dict()
    def of(self, x):
        if x not in self.table:
            self.table[x] = randint(0,1000)
        return self.table[x]
class HashTablesList:
    def __init__(self):
        self.tables = {}
    def get(self, l):
        if l not in self.tables:
            return None
        return self.tables[l]
    def insert(self, l):
        self.tables[l] = {'table': defaultdict(set), 'random_function': RandomHashInt() }

In [134]:
def build_index(S, q = 2):
    """
    Input: set of input strings S = {s1, . . . , sn }
    Output: set of hash tables {(Hi, fi )}, where for the i-th hash table, each
    string s is hashed into the fi(s)-th bucket of Hi
    """
    hash_tables = HashTablesList()
    for i,si in enumerate(S):
        Ri = Rank(si, 0, q)
        Pi = Partition(si, Ri, 0, len(Ri)-1)
        for (ssub, l) in Pi:
            if hash_tables.get(l) is None:
                hash_tables.insert(l)
            fl = hash_tables.get(l)['random_function']
            hash_tables.get(l)['table'][(ssub, fl.of(ssub))].add(i)
    return hash_tables

In [135]:
S = ['ACGTTCGACTGGTTAG',
     'CCGTTCGAACTGGTTAG',
     'ACATTCGACTGGTTGAG',
     'TCGAACGTTCGAACGT']
index = build_index(S, 3)

In [136]:
index.tables

{4: {'table': defaultdict(set,
              {('ACGTTCGAC', 566): {0}, ('', 545): {0}, ('TGGTT', 877): {0}}),
  'random_function': <__main__.RandomHashInt at 0x7f4eb44d5160>},
 3: {'table': defaultdict(set,
              {('ACG', 136): {0},
               ('', 846): {0, 1, 2, 3},
               ('TTCGAC', 283): {0},
               ('AACT', 701): {1},
               ('GGTT', 223): {1},
               ('GACT', 28): {2},
               ('GGTTG', 276): {2},
               ('TCG', 49): {3},
               ('AACGTTCGAAC', 939): {3}}),
  'random_function': <__main__.RandomHashInt at 0x7f4eb44d58e0>},
 0: {'table': defaultdict(set,
              {('A', 424): {0, 1, 2, 3},
               ('C', 489): {0, 1, 2, 3},
               ('G', 690): {0, 1, 2, 3},
               ('T', 746): {0, 1, 2, 3},
               ('', 807): {0, 1, 2, 3}}),
  'random_function': <__main__.RandomHashInt at 0x7f4eb44d5e50>},
 1: {'table': defaultdict(set,
              {('TTC', 157): {0},
               ('', 613): {0, 1

In [137]:
def ED(s1, s2):
    # Edit distance between two strings. Levenshtein distance, in this case
    # https://stackoverflow.com/questions/2460177/edit-distance-in-python
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [138]:
ED('abc', 'cba')

2

In [139]:
def r(s, alpha, K, q = 2):
    import math
    return max(0, math.floor((len(s)-q+1-alpha*K)/(2*alpha*K + 2)))

def min_search_threshold(S, t, K):
    """
    Input: set of strings S = {s1, . . . , sn }; query string t; distance threshold K
    Output: O ← {i | si ∈ S; ED(si, t) ≤ K }
    """
    tables = build_index(S).tables
    O = []
    C = []
    alpha = 120
    R = Rank(t, r(t, alpha, K))
    P = Partition(t, R, 0, len(R)-1)
    for (tsub, l) in P:
        fl = tables[l]['random_function']
        for i in tables[l]['table'][(tsub, fl.of(tsub))]:
            if abs(len(S[i])-len(t)) <= K:
                C.append((i,S[i]))
    C = list(set(C))
    for i,si in C:
        if ED(si,t) <= K:
            O.append(i)
    return O

In [140]:
S = ['ACGTTCGACTGGTTAG',
     'CCGTTCGAACTGGTTAG',
     'ACATTCGACTGGTTGAG',
     'TCGAACGTTCGAACGT']
min_search_threshold(S, 'ACGTTCGACTGG', 5)

[0]